In [2]:
pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("cluster").getOrCreate()
print('Spark Version: {}'.format(spark.version))

Spark Version: 3.4.1


In [6]:
df = pd.read_csv("Acoustic_Extinguisher_Fire_Dataset.csv")
df["FUEL"] = df["FUEL"].astype("category")
df["FUEL"] = df["FUEL"].cat.codes
x = df.iloc[:,:-1]
y = df.iloc[:,-1]
df.head()

,SIZE,FUEL,DISTANCE,DESIBEL,AIRFLOW,FREQUENCY,STATUS
0,1,0,10,96,0.0,75,0
1,1,0,10,96,0.0,72,1
2,1,0,10,96,2.6,70,1
3,1,0,10,96,3.2,68,1
4,1,0,10,109,4.5,67,1


In [11]:
from pyspark.ml.feature import VectorAssembler
  
vec_assembler = VectorAssembler(inputCols = df.columns,
                                outputCol='features')
  
final_data = vec_assembler.transform(dataset)
final_data.select('features').show(5)

+--------------------+
|            features|
+--------------------+
|[2.0,2.0,3.0,3.0,...|
|[2.0,2.0,3.0,3.0,...|
|[2.0,2.0,2.0,3.0,...|
|[1.0,1.0,1.0,3.0,...|
|[2.0,2.0,1.0,3.0,...|
+--------------------+
only showing top 5 rows



In [7]:
cl = df.columns[:-1]
cl = list(cl)

In [12]:
from pyspark.ml.feature import StandardScaler
  
scaler = StandardScaler(inputCol="features", 
                        outputCol="scaledFeatures")
  
scalerModel = scaler.fit(final_data)
final_data = scalerModel.transform(final_data)
  
final_data.select('scaledFeatures').show(5)

+--------------------+
|      scaledFeatures|
+--------------------+
|[3.26182317820262...|
|[3.26182317820262...|
|[3.26182317820262...|
|[1.63091158910131...|
|[3.26182317820262...|
+--------------------+
only showing top 5 rows



In [8]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
df[cl] = ss.fit_transform(df[cl])

In [9]:
df1 = spark.createDataFrame(df)
df1.head()

Row(SIZE=-1.3774219384009612, FUEL=-1.1852755159795074, DISTANCE=-1.6431676725154982, DESIBEL=-0.0464415350316309, AIRFLOW=-1.4728852648142814, FREQUENCY=2.0722013081031, STATUS=0)

In [13]:
from pyspark.ml.feature import VectorAssembler, PCA
  
vec_assembler = VectorAssembler(inputCols = cl,
                                outputCol='features')

In [11]:
final_data = vec_assembler.transform(df1)
final_data.select(['features','STATUS']).show(5)

+--------------------+------+
|            features|STATUS|
+--------------------+------+
|[-1.3774219384009...|     0|
|[-1.3774219384009...|     1|
|[-1.3774219384009...|     1|
|[-1.3774219384009...|     1|
|[-1.3774219384009...|     1|
+--------------------+------+
only showing top 5 rows



In [12]:
train_data, test_data = final_data.randomSplit([0.8, 0.2], seed = 42)

In [17]:
pca = PCA(k=3, inputCol='features', outputCol = 'pca')
model = pca.fit(final_data)
model.setOutputCol("Output")
model.transform(final_data)
model.explainedVariance

DenseVector([0.3221, 0.2444, 0.186])